In [2]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

#import missingno as msno

%matplotlib inline

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image
import numpy.matlib

from numpy import *

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
# Reading price data from stocks 
temp_csv = Path("Resources/s_PX.csv")
s_price=pd.read_csv(temp_csv)
s_price.set_index(pd.to_datetime(s_price['Date'], infer_datetime_format=True), inplace=True)
s_price.drop(columns=['Date'], inplace=True)
s_price = s_price.dropna(how='all')
s_price.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-02,NaN,14.7333,10.5489,6.3683,2.4012,NaN,NaN,NaN,NaN,19.4629,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-05,NaN,14.5639,10.6810,6.5333,2.6207,NaN,NaN,NaN,NaN,19.4904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Reading price data from stocks - monthly
#s_price_m=s_price.copy()
#s_price_m.reset_index(inplace=True)
#s_price_m.Date = pd.to_datetime(s_price_m.Date)
#s_price_m=s_price_m.resample('m', on='Date').last()
#s_price_m.drop(columns=['Date'], inplace=True)
#s_price_m.head(2)

In [6]:
# Reading price data from stocks - monthly
s_price_m=s_price.copy()
s_price_m=s_price_m.groupby([s_price_m.index.year, s_price_m.index.month]).tail(1)
s_price_m.head(5)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,14.5639,11.7944,6.4673,2.5933,NaN,NaN,NaN,NaN,18.3527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,14.9704,13.4928,6.5993,2.7648,NaN,NaN,NaN,NaN,19.7508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,NaN,15.9187,13.6024,7.5562,2.9569,NaN,NaN,NaN,NaN,20.1346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-04-30,NaN,16.3001,15.5483,7.4572,3.1833,NaN,NaN,NaN,NaN,22.4168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-05-29,NaN,16.1292,16.3229,8.4141,2.6207,NaN,NaN,NaN,NaN,22.5542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Calculating stock returns 
s_ret = s_price.pct_change()
s_ret = s_ret.dropna(how='all')
s_ret.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-05,NaN,-0.011498,0.012523,0.025910,0.091413,NaN,NaN,NaN,NaN,0.001413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-06,NaN,-0.009297,-0.005299,0.010102,0.000000,NaN,NaN,NaN,NaN,-0.020395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Calculating stock returns for y label 
s_ret_m = s_price_m.pct_change()
s_ret_m=s_ret_m.shift(periods=-1)
s_ret_m.tail(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-29,0.019267,0.031548,0.001204,-0.020087,0.055422,0.0,0.019323,-0.004144,-0.02118,0.006494,...,-0.013305,-0.027874,0.005669,0.009719,-0.010032,0.036763,0.018752,-0.02778,-0.016089,-0.017366
2019-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#calculating y label for ML model 1 means outperformer -1 means underperformer and 0 are neutral stocks we should not use for trainign 
#df=s_ret_m.quantile(2/3, axis=1 ).to_frame()
#df=(df[[col for col in df.columns for i in range(np.size(s_ret_m,1))]])
#df = pd.np.divide(s_ret_m, df)
#df=df>1
#df *= 1
#df.head(2)
#df2=s_ret_m.quantile(1/3, axis=1 ).to_frame()
#df2=(df2[[col for col in df2.columns for i in range(np.size(s_ret_m,1))]])
#df2 = pd.np.divide(s_ret_m, df2)
#df2=df2<1
#df2 *= 1
#df2=-df2
#df2.head(2)
#df_ylabel=df+df2
#df_ylabel.head(2)
#df_ylabel.sum(df_ylabel)
#(df == 1).astype(int).sum(axis=1)


In [25]:
#calculating y label for ML model 1 means outperformer -1 means underperformer and 0 are neutral stocks we should not use for trainign 
df=s_ret_m.quantile(2/3, axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_ret_m,1))]])  #repeating value for all the columns 
#df=s_ret_m>df
df = pd.np.greater(s_ret_m, df)
df *= 1
df.columns = s_ret_m.columns
df.head(2)
df2=s_ret_m.quantile(1/3, axis=1).to_frame()
df2=(df2[[col for col in df2.columns for i in range(np.size(s_ret_m,1))]])
#df2=s_ret_m<df2
df2 = pd.np.greater(df2, s_ret_m)
df2 *= 1
df2=-df2
df2.columns = s_ret_m.columns
df2.head(2)
df_ylabel=df+df2
df_ylabel.head(2)
#df_ylabel.sum(df_ylabel)
#(df2 == -1).astype(int).sum(axis=1)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,0,-1,1,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998-02-27,0,0,-1,1,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
np.size(df_ylabel,0)

264

In [26]:
# Reading market cap data from stocks 
temp_csv = Path("Resources/s_MarketCap.csv")
s_mkt=pd.read_csv(temp_csv)
s_mkt.set_index(pd.to_datetime(s_mkt['Date'], infer_datetime_format=True), inplace=True)
s_mkt.drop(columns=['Date'], inplace=True)
s_mkt = s_mkt.dropna(how='all')
s_mkt.head(2) 

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,10399.3620,15436.0545,1577.9225,1327.6778,NaN,NaN,NaN,NaN,39031.8495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,10689.5767,17658.8463,1610.1250,1421.9855,NaN,NaN,NaN,NaN,42005.1495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_DebttoEquity.csv")
s_fund1=pd.read_csv(temp_csv)
s_fund1.set_index(pd.to_datetime(s_fund1['Date'], infer_datetime_format=True), inplace=True)
s_fund1.drop(columns=['Date'], inplace=True)
s_fund1 = s_fund1.dropna(how='all')
s_fund1.head(2)

FileNotFoundError: [Errno 2] File b'Resources/s_DebttoEquity.csv' does not exist: b'Resources/s_DebttoEquity.csv'

In [12]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_NetIncomeMargin.csv")
s_fund2=pd.read_csv(temp_csv)
s_fund2.set_index(pd.to_datetime(s_fund2['Date'], infer_datetime_format=True), inplace=True)
s_fund2.drop(columns=['Date'], inplace=True)
s_fund2 = s_fund2.dropna(how='all')
s_fund2.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
2001-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_FreeCashFlowEquity.csv")
s_fund3=pd.read_csv(temp_csv)
s_fund3.set_index(pd.to_datetime(s_fund3['Date'], infer_datetime_format=True), inplace=True)
s_fund3.drop(columns=['Date'], inplace=True)
s_fund3 = s_fund3.dropna(how='all')
s_fund3.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,1568.5819,405.852,-111.346,17.905,NaN,NaN,-4.845,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,1568.5819,405.852,-111.346,17.905,NaN,NaN,-4.845,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_GrossMargin.csv")
s_fund4=pd.read_csv(temp_csv)
s_fund4.set_index(pd.to_datetime(s_fund4['Date'], infer_datetime_format=True), inplace=True)
s_fund4.drop(columns=['Date'], inplace=True)
s_fund4 = s_fund4.dropna(how='all')
s_fund4.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
2003-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_PB.csv")
s_fund5=pd.read_csv(temp_csv)
s_fund5.set_index(pd.to_datetime(s_fund5['Date'], infer_datetime_format=True), inplace=True)
s_fund5.drop(columns=['Date'], inplace=True)
s_fund5 = s_fund5.dropna(how='all')
s_fund5.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,0.9902,6.5864,0.3880,4.219,NaN,NaN,NaN,NaN,2.6822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,1.0178,7.5348,0.3959,4.498,NaN,NaN,NaN,NaN,2.8865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_ROE.csv")
s_fund6=pd.read_csv(temp_csv)
s_fund6.set_index(pd.to_datetime(s_fund6['Date'], infer_datetime_format=True), inplace=True)
s_fund6.drop(columns=['Date'], inplace=True)
s_fund6 = s_fund6.dropna(how='all')
s_fund6.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,11.7408,32.9726,-5.8446,28.6663,NaN,NaN,19.089,NaN,21.9976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,11.7408,32.9726,-5.8446,28.6663,NaN,NaN,19.089,NaN,21.9976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_ROA.csv")
s_fund7=pd.read_csv(temp_csv)
s_fund7.set_index(pd.to_datetime(s_fund7['Date'], infer_datetime_format=True), inplace=True)
s_fund7.drop(columns=['Date'], inplace=True)
s_fund7 = s_fund7.dropna(how='all')
s_fund7.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,2.8192,17.9028,-1.2286,17.8698,NaN,NaN,15.3539,NaN,1.7426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,2.8192,17.9028,-1.2286,17.8698,NaN,NaN,15.3539,NaN,1.7426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Reading fundamental data for stocks 
temp_csv = Path("Resources/s_PE.csv")
s_fund8=pd.read_csv(temp_csv)
s_fund8.set_index(pd.to_datetime(s_fund8['Date'], infer_datetime_format=True), inplace=True)
s_fund8.drop(columns=['Date'], inplace=True)
s_fund8 = s_fund8.dropna(how='all')
s_fund8.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,8.3701,20.4134,NaN,17.4336,NaN,NaN,NaN,NaN,13.2351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,8.6037,23.3530,NaN,18.5866,NaN,NaN,NaN,NaN,14.2433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
##Building monthly feature set for stocks 
window_year=252
window_month=21
window_quarter=63

s_feat1=s_price/s_price.shift(window_year)-1 #Price momentum 12M 
s_feat2=s_price.shift(window_month)/s_price.shift(window_year)-1 #Price momentum 12M-1M 
s_feat3=s_price/s_price.rolling(window_year).max() #Price to 52W High 
s_feat4=s_price/s_price.rolling(window_year).min() #Price to 52W low 
s_feat5=s_price/s_price.shift(window_month)-1 #Return 1M
s_feat6=s_price/s_price.shift(window_quarter)-1 #Return 1Q
s_feat7=s_price/s_price.shift(window_year*5)-1 #Return 60M
s_feat8=s_ret.rolling(window_month).max() #Maximum daily return over 1M
s_feat9=s_ret.rolling(window_month).std() #Realized vol 1M
s_feat10=s_ret.rolling(window_quarter).std() #Realized vol 3M
s_feat11=np.log(s_mkt) #Log Market capitalization - already monthly
s_feat10.tail()

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-04,0.019951,0.018801,0.025619,0.007981,0.011457,0.0,0.012812,0.020318,0.017863,0.010602,...,0.023704,0.022899,0.022080,0.016341,0.015193,0.033261,0.012631,0.011306,0.013774,0.019711
2019-12-05,0.019917,0.018799,0.025680,0.008079,0.010034,0.0,0.012874,0.020317,0.017860,0.010598,...,0.023703,0.022895,0.022112,0.016386,0.015235,0.033232,0.012632,0.011221,0.013776,0.019702
2019-12-06,0.019955,0.018798,0.024285,0.008171,0.010015,0.0,0.012784,0.019717,0.017126,0.010884,...,0.023591,0.022762,0.022041,0.016400,0.015094,0.032687,0.012651,0.011265,0.013270,0.019704
2019-12-09,0.019557,0.018574,0.024105,0.008204,0.010023,0.0,0.012289,0.019738,0.016319,0.010517,...,0.023545,0.022485,0.021411,0.015829,0.014946,0.032690,0.012138,0.011272,0.013161,0.018994
2019-12-10,0.018630,0.018580,0.023389,0.008201,0.010803,0.0,0.012264,0.019725,0.016444,0.010503,...,0.023553,0.022264,0.021385,0.015639,0.014962,0.032596,0.012003,0.011280,0.013159,0.018993


In [20]:
# monthly features here 123
#s_feature

In [21]:
# # monthly features - not using resample
s_feat1=s_feat1.groupby([s_feat1.index.year, s_feat1.index.month]).tail(1)
s_feat2=s_feat2.groupby([s_feat2.index.year, s_feat2.index.month]).tail(1)
s_feat3=s_feat3.groupby([s_feat3.index.year, s_feat3.index.month]).tail(1)
s_feat4=s_feat4.groupby([s_feat4.index.year, s_feat4.index.month]).tail(1)
s_feat5=s_feat5.groupby([s_feat5.index.year, s_feat5.index.month]).tail(1)
s_feat6=s_feat6.groupby([s_feat6.index.year, s_feat6.index.month]).tail(1)
s_feat7=s_feat7.groupby([s_feat7.index.year, s_feat7.index.month]).tail(1)
s_feat8=s_feat8.groupby([s_feat8.index.year, s_feat8.index.month]).tail(1)
s_feat9=s_feat9.groupby([s_feat9.index.year, s_feat9.index.month]).tail(1)
s_feat10=s_feat10.groupby([s_feat10.index.year, s_feat10.index.month]).tail(1)
s_feat10.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#normalizing monthly features from 0 to 1 
#Feature 1 
rank_f=s_feat1.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat1,1))]])
s_feat1 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat2.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat2,1))]])
s_feat2 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat3.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat3,1))]])
s_feat3 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat4.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat4,1))]])
s_feat4 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat5.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat5,1))]])
s_feat5 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat6.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat6,1))]])
s_feat6 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat7.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat7,1))]])
s_feat7 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat8.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat8,1))]])
s_feat8 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat9.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat9,1))]])
s_feat9 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat10.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat10,1))]])
s_feat10 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_feat11.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_feat11,1))]])
s_feat11 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund1.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund1,1))]])
s_fund1 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund2.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund2,1))]])
s_fund2 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund3.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund3,1))]])
s_fund3 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund4.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund4,1))]])
s_fund4 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund5.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund5,1))]])
s_fund5 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund6.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund6,1))]])
s_fund6 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund7.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund7,1))]])
s_fund7 = pd.np.divide(rank_f, df)
#Feature 
rank_f=s_fund8.rank(axis=1)
df=rank_f.max(axis=1).to_frame()
df=(df[[col for col in df.columns for i in range(np.size(s_fund8,1))]])
s_fund8 = pd.np.divide(rank_f, df)

s_fund8.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,0.181818,0.777778,NaN,0.673401,NaN,NaN,NaN,NaN,0.468013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,0.172297,0.814189,NaN,0.662162,NaN,NaN,NaN,NaN,0.466216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
np.size(s_fund8)

132815

In [24]:
s_fund8.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,0.181818,0.777778,NaN,0.673401,NaN,NaN,NaN,NaN,0.468013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,0.172297,0.814189,NaN,0.662162,NaN,NaN,NaN,NaN,0.466216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
s_fund1.head(2)

,ILMN UW Equity,EIX UN Equity,GPS UN Equity,ES UN Equity,NLOK UW Equity,AET UN Equity,MCO UN Equity,RMD UN Equity,LYB UN Equity,AXP UN Equity,...,ALGN UW Equity,ADS UN Equity,LKQ UW Equity,NLSN UN Equity,GRMN UW Equity,XEC UN Equity,ZTS UN Equity,DLR UN Equity,EQIX UW Equity,DISCK UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,0.882184,0.376437,0.873563,0.132184,NaN,NaN,0.077586,NaN,0.948276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,0.885057,0.376437,0.876437,0.135057,NaN,NaN,0.080460,NaN,0.948276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Organizing data for ML model 
x=df_ylabel.copy()
y_stacked = x.stack() 
y_stacked=y_stacked.to_frame()
y_stacked.rename(columns={0: 'ylabel'}, inplace=True)
y_stacked.head()

x1=s_fund1.copy()
df_stacked1 = x1.stack() 
df_stacked1=df_stacked1.to_frame()
df_stacked1.rename(columns={0: 'feature_1'}, inplace=True)
df_stacked1.head()

x2=s_fund2.copy()
df_stacked2 = x2.stack() 
df_stacked2=df_stacked2.to_frame()
df_stacked2.rename(columns={0: 'feature_2'}, inplace=True)
df_stacked2.head()

x3=s_fund3.copy()
df_stacked3 = x3.stack() 
df_stacked3=df_stacked3.to_frame()
df_stacked3.rename(columns={0: 'feature_3'}, inplace=True)
df_stacked3.head()

x4=s_fund4.copy()
df_stacked4 = x4.stack() 
df_stacked4=df_stacked4.to_frame()
df_stacked4.rename(columns={0: 'feature_4'}, inplace=True)
df_stacked4.head()

x5=s_fund5.copy()
df_stacked5 = x5.stack() 
df_stacked5=df_stacked5.to_frame()
df_stacked5.rename(columns={0: 'feature_5'}, inplace=True)
df_stacked5.head()

x6=s_fund6.copy()
df_stacked6 = x6.stack() 
df_stacked6=df_stacked6.to_frame()
df_stacked6.rename(columns={0: 'feature_6'}, inplace=True)
df_stacked6.head()

x7=s_fund7.copy()
df_stacked7 = x7.stack() 
df_stacked7=df_stacked7.to_frame()
df_stacked7.rename(columns={0: 'feature_7'}, inplace=True)
df_stacked7.head()

x8=s_fund8.copy()
df_stacked8 = x8.stack() 
df_stacked8=df_stacked8.to_frame()
df_stacked8.rename(columns={0: 'feature_8'}, inplace=True)
df_stacked8.head()

#Technical 
x9=s_feat1.copy()
df_stacked9 = x9.stack() 
df_stacked9=df_stacked9.to_frame()
df_stacked9.rename(columns={0: 'feature_9'}, inplace=True)
df_stacked9.head()

x10=s_feat2.copy()
df_stacked10 = x10.stack() 
df_stacked10=df_stacked10.to_frame()
df_stacked10.rename(columns={0: 'feature_10'}, inplace=True)
df_stacked10.head()

x11=s_feat3.copy()
df_stacked11 = x11.stack() 
df_stacked11=df_stacked11.to_frame()
df_stacked11.rename(columns={0: 'feature_11'}, inplace=True)
df_stacked11.head()

x12=s_feat4.copy()
df_stacked12 = x12.stack() 
df_stacked12=df_stacked12.to_frame()
df_stacked12.rename(columns={0: 'feature_12'}, inplace=True)
df_stacked12.head()

x13=s_feat5.copy()
df_stacked13 = x13.stack() 
df_stacked13=df_stacked13.to_frame()
df_stacked13.rename(columns={0: 'feature_13'}, inplace=True)
df_stacked13.head()

x14=s_feat6.copy()
df_stacked14 = x14.stack() 
df_stacked14=df_stacked14.to_frame()
df_stacked14.rename(columns={0: 'feature_14'}, inplace=True)
df_stacked14.head()

x15=s_feat7.copy()
df_stacked15 = x15.stack() 
df_stacked15=df_stacked15.to_frame()
df_stacked15.rename(columns={0: 'feature_15'}, inplace=True)
df_stacked15.head()

x16=s_feat8.copy()
df_stacked16 = x16.stack() 
df_stacked16=df_stacked16.to_frame()
df_stacked16.rename(columns={0: 'feature_16'}, inplace=True)
df_stacked16.head()

x17=s_feat9.copy()
df_stacked17 = x17.stack() 
df_stacked17=df_stacked17.to_frame()
df_stacked17.rename(columns={0: 'feature_17'}, inplace=True)
df_stacked17.head()

x18=s_feat10.copy()
df_stacked18 = x18.stack() 
df_stacked18=df_stacked18.to_frame()
df_stacked18.rename(columns={0: 'feature_18'}, inplace=True)
df_stacked18.head()

x19=s_feat11.copy()
df_stacked19 = x19.stack() 
df_stacked19=df_stacked19.to_frame()
df_stacked19.rename(columns={0: 'feature_19'}, inplace=True)
df_stacked19.head()

feature_19
Date                                 
1998-01-30 EIX UN Equity     0.739521
           GPS UN Equity     0.820359
           ES UN Equity      0.272455
           NLOK UW Equity    0.239521
           AXP UN Equity     0.913174

In [27]:
#Generating feature set for ML 
feature_set=pd.concat([y_stacked,df_stacked1,df_stacked2,df_stacked3,df_stacked4,df_stacked5,df_stacked6,df_stacked7,df_stacked8,df_stacked9,df_stacked10,df_stacked11,df_stacked12,df_stacked13,df_stacked14,df_stacked15,df_stacked16,df_stacked17,df_stacked18,df_stacked19],axis="columns", join="outer")
feature_set.tail(10)


ylabel  feature_1  feature_2  feature_3  feature_4  \
Date                                                                            
2019-12-10 XL UN Equity         0        NaN        NaN        NaN        NaN   
           XLNX UW Equity       0        NaN        NaN        NaN        NaN   
           XOM UN Equity        0        NaN        NaN        NaN        NaN   
           XRAY UW Equity       0        NaN        NaN        NaN        NaN   
           XRX UN Equity        0        NaN        NaN        NaN        NaN   
           XYL UN Equity        0        NaN        NaN        NaN        NaN   
           YUM UN Equity        0        NaN        NaN        NaN        NaN   
           ZBH UN Equity        0        NaN        NaN        NaN        NaN   
           ZION UW Equity       0        NaN        NaN        NaN        NaN   
           ZTS UN Equity        0        NaN        NaN        NaN        NaN   

                           feature_5  feature_6  feature_7  feature_8  \
Date                                                                    
2019-12-10 XL UN Equity          NaN        NaN        NaN        NaN   
           XLNX UW Equity        NaN        NaN        NaN        NaN   
           XOM UN Equity         NaN        NaN        NaN        NaN   
           XRAY UW Equity        NaN        NaN        NaN        NaN   
           XRX UN Equity         NaN        NaN        NaN        NaN   
           XYL UN Equity         NaN        NaN        NaN        NaN   
           YUM UN Equity         NaN        NaN        NaN        NaN   
           ZBH UN Equity         NaN        NaN        NaN        NaN   
           ZION UW Equity        NaN        NaN        NaN        NaN   
           ZTS UN Equity         NaN        NaN        NaN        NaN   

                           feature_9  feature_10  feature_11  feature_12  \
Date                                                                       
2019-12-10 XL UN Equity          NaN         NaN         NaN         NaN   
           XLNX UW Equity   0.271222    0.368313    0.046958    0.153209   
           XOM UN Equity    0.144928    0.172840    0.213447    0.060041   
           XRAY UW Equity   0.956522    0.967078    0.723586    0.881988   
           XRX UN Equity    0.933747    0.973251    0.480256    0.987578   
           XYL UN Equity    0.447205    0.518519    0.326574    0.283644   
           YUM UN Equity    0.378882    0.374486    0.185699    0.138716   
           ZBH UN Equity    0.741201    0.746914    0.727855    0.774327   
           ZION UW Equity   0.436853    0.502058    0.787620    0.440994   
           ZTS UN Equity    0.830228    0.781893    0.586980    0.830228   

                           feature_13  feature_14  feature_15  feature_16  \
Date                                                                        
2019-12-10 XL UN Equity           NaN         NaN         NaN    0.019802   
           XLNX UW Equity    0.134576    0.047619    0.765591    0.643564   
           XOM UN Equity     0.219462    0.244306    0.161290    0.681188   
           XRAY UW Equity    0.370600    0.838509    0.210753    0.136634   
           XRX UN Equity     0.101449    0.939959    0.255914    0.768317   
           XYL UN Equity     0.242236    0.209110    0.754839    0.196040   
           YUM UN Equity     0.453416    0.051760    0.677419    0.388119   
           ZBH UN Equity     0.461698    0.505176    0.335484    0.851485   
           ZION UW Equity    0.271222    0.861284    0.636559    0.330693   
           ZTS UN Equity     0.757764    0.430642    0.926882    0.235644   

                           feature_17  feature_18  feature_19  
Date                                                           
2019-12-10 XL UN Equity      0.033663    0.031683         NaN  
           XLNX UW Equity    0.708911    0.801980         NaN  
           XOM UN Equity     0.463366    0.461386         NaN  
           XRAY UW 

In [28]:
#feature_set.index
np.size(feature_set,0)

133320

In [29]:
feature_set_copy=feature_set.copy()
feature_set_copy.reset_index(inplace=True)
feature_set_copy.head()

,Date,level_1,ylabel,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
0,1998-01-30,A UN Equity,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-01-30,AAL UW Equity,0,0.614943,NaN,0.023891,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-01-30,AAP UN Equity,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998-01-30,AAPL UW Equity,1,0.632184,NaN,0.798635,NaN,0.419355,0.019108,0.028302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.341317
4,1998-01-30,ABBV UN Equity,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#df_pred=pd.DataFrame(columns=["Prediction", "Actual" , "Probability"])
#appended_data = []
appended_data = pd.DataFrame(columns=['Prediction', 'Actual', 'Probability', 'Reference'])
feat = []
#x=0
for x in range(248):   ## max range is 251 --248
    start_train=(x)*505
    end_train=505*(x+12)
    end_test=505*(x+13)
    train=feature_set_copy[start_train:end_train]
    train.drop(columns=['Date','level_1'], inplace=True) #Droping dates and tickers 
    train["ylabel"].where((train["ylabel"]!=0) ,np.NaN,inplace=True) #Removing ylabel =0 to Nan
    train = train[np.isfinite(train['ylabel'])]
    train=train.fillna(0.5)  #filling missing values with 0.5 
    #print(train)
    test=feature_set_copy[end_train:end_test]
    test.drop(columns=['Date','level_1'], inplace=True) #Droping dates and tickers 
    test["ylabel"].where((test["ylabel"]!=0) ,np.NaN,inplace=True) #Removing ylabel =0 to Nan
    test = test[np.isfinite(test['ylabel'])]
    test=test.fillna(0.5)  #filling missing values with 0.5 
    #print(test)
    y_train = train["ylabel"].values.reshape(-1, 1)
    X_train_scaled=train.copy()  
    X_train_scaled.drop(columns=['ylabel'], inplace=True) #Droping dates and tickers

    y_test = test["ylabel"].values.reshape(-1, 1)
    X_test_scaled=test.copy()  
    X_test_scaled.drop(columns=['ylabel'], inplace=True) #Droping dates and tickers

    learning_rate=1   # test with several parameters [0.05, 0.1, 0.25, 0.5, 0.75, 1] 
    classifier = GradientBoostingClassifier(n_estimators=20,
                                                learning_rate=learning_rate,
                                                max_features=5,
                                                max_depth=3,
                                                random_state=0)
    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
        # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()

    # Make Prediction
    predictions = classifier.predict(X_test_scaled)
    #pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(3)

    # Make Prediction
    predictions_prob = classifier.predict_proba(X_test_scaled)
    #pd.DataFrame({"Prediction_pro": predictions_prob, "Actual": y_test.ravel()}).head(3)
    
    df_pred=pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel() , "Probability": predictions_prob[:, 1]})
    df_pred.index=test.index
    df_pred['Reference'] = df_pred.index
    #df_pred.drop(columns=['index'], inplace=True)
    #df_pred.reset_index(inplace=True)
    
    #print(df_pred.head(2))
    
    #appended_data.append(df_pred)
    appended_data = pd.concat([appended_data,df_pred])
    #feat=feat.append(classifier.feature_importances_)
    temp=classifier.feature_importances_
    feat.append(temp)
    


Accuracy score (training): 0.708
Accuracy score (validation): 0.534

Accuracy score (training): 0.708
Accuracy score (validation): 0.483

Accuracy score (training): 0.715
Accuracy score (validation): 0.428

Accuracy score (training): 0.715
Accuracy score (validation): 0.460

Accuracy score (training): 0.727
Accuracy score (validation): 0.462

Accuracy score (training): 0.705
Accuracy score (validation): 0.475

Accuracy score (training): 0.708
Accuracy score (validation): 0.537

Accuracy score (training): 0.711
Accuracy score (validation): 0.521

Accuracy score (training): 0.719
Accuracy score (validation): 0.606

Accuracy score (training): 0.714
Accuracy score (validation): 0.576

Accuracy score (training): 0.719
Accuracy score (validation): 0.520

Accuracy score (training): 0.707
Accuracy score (validation): 0.520

Accuracy score (training): 0.705
Accuracy score (validation): 0.527

Accuracy score (training): 0.724
Accuracy score (validation): 0.437

Accuracy score (training): 0.714
A

In [31]:
#Seeing feature importance table thoughout the entire period - The higher the number the more important the feature 
feat = pd.DataFrame(feat)
feat.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.060014,0.0,0.081026,0.0,0.152345,0.045494,0.094555,0.067083,0.000000,0.000000,0.013361,0.034252,0.117956,0.061586,0.0,0.078736,0.030982,0.056362,0.106249
1,0.103091,0.0,0.088540,0.0,0.113170,0.054860,0.044524,0.047001,0.038724,0.022665,0.051232,0.031870,0.037150,0.096672,0.0,0.083496,0.049379,0.049811,0.087814


In [32]:
#df_pred.reset_index

df_pred.head()

,Prediction,Actual,Probability,Reference
130795,1.0,1.0,0.710582,130795
130797,-1.0,1.0,0.230899,130797
130798,1.0,1.0,0.557991,130798
130799,1.0,1.0,0.916624,130799
130800,-1.0,-1.0,0.291165,130800


In [33]:
print(classifier.feature_importances_)

[0.05240178 0.03664282 0.02637497 0.02453286 0.02635017 0.02407996
 0.07626455 0.06263103 0.07231706 0.05504137 0.01809236 0.04709283
 0.10648644 0.0617165  0.06764332 0.07290115 0.0731489  0.06401929
 0.03226263]


In [34]:
appended_data.tail()

,Prediction,Actual,Probability,Reference
131291,1.0,-1.0,0.520852,131291
131296,1.0,-1.0,0.780340,131296
131297,1.0,-1.0,0.564248,131297
131298,-1.0,1.0,0.393057,131298
131299,1.0,-1.0,0.504314,131299


In [35]:
#appended_data.size

In [36]:
#df_pred.size

In [37]:
signal_input=pd.concat([feature_set_copy,appended_data],axis="columns", join="outer")
signal_input.drop(columns=['feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15','feature_16','feature_17','feature_18','feature_19'], inplace=True)
signal_input.head()

,Date,level_1,ylabel,Prediction,Actual,Probability,Reference
0,1998-01-30,A UN Equity,0,NaN,NaN,NaN,NaN
1,1998-01-30,AAL UW Equity,0,NaN,NaN,NaN,NaN
2,1998-01-30,AAP UN Equity,0,NaN,NaN,NaN,NaN
3,1998-01-30,AAPL UW Equity,1,NaN,NaN,NaN,NaN
4,1998-01-30,ABBV UN Equity,0,NaN,NaN,NaN,NaN


In [38]:
signal_input_mat=signal_input.copy()
signal_input_mat.drop(columns=['ylabel','Prediction','Actual','Reference'], inplace=True)
signal_input_mat=signal_input_mat.set_index(['Date', 'level_1'])
signal_input_mat=signal_input_mat.unstack()
#signal_input_mat=signal_input_mat.droplevel(level=1)
signal_input_mat=signal_input_mat.droplevel(0, axis=1) 
signal_input_mat=signal_input_mat.shift(1)
signal_input_mat.head(2)

level_1,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,ADBE UW Equity,...,XL UN Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#Selecting top N stocks per month to go long. 
N = 20
df3 = signal_input_mat.mask(signal_input_mat.rank(axis=1, method='min', ascending=False) > N, 0) #taking top 20 stocks     
df3[df3.isnull()] = 0   # nan
df3[df3 > 0] = 1/N
df3.head(2)
#(df3!=0).sum(axis=1)  #to check 
#df3.sum(axis=1) #to check

level_1,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,ADBE UW Equity,...,XL UN Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
#(df3!=0).sum(axis=1)  #to check 

In [41]:
df4 = pd.np.multiply(s_ret_m, df3)
df4[df4.isnull()] = 0   # nan
df4=df4.sum(axis=1).to_frame()
df4.head(2)

,0
Date,
1998-01-30,0.0
1998-02-27,0.0


In [42]:
#pnl=(1 + df4).cumprod() 
#pnl=pnl.rename(columns={0: "ML Strategy PnL"})
#pnl.head(2)
#type(pnl)
#list(pnl.columns) 
#pnl.plot()

In [43]:
# Reading price data from SP500 index 
temp_csv = Path("Resources/prices.csv")
df_price=pd.read_csv(temp_csv)
df_price.set_index(pd.to_datetime(df_price['Date'], infer_datetime_format=True), inplace=True)
df_price.drop(columns=['Date','XLE US Equity', 'XLF US Equity', 'XLI US Equity',
       'XLK US Equity', 'XLP US Equity', 'XLRE US Equity', 'XLU US Equity',
       'XLV US Equity', 'XLY US Equity','XLB US Equity'], inplace=True)
df_price.head(2)

,SPXT Index
Date,
1994-01-03,567.09
1994-01-04,569.07


In [44]:
# Reading price data from stocks - monthly
s_price_m=df_price.copy()
s_price_m=s_price_m.groupby([s_price_m.index.year, s_price_m.index.month]).tail(1)
s_price_m.head(2)

#Calculating stock returns 
ret_sp = s_price_m.pct_change()
ret_sp = ret_sp.dropna(how='all')
ret_sp.head(2)

,SPXT Index
Date,
1994-02-28,-0.027148
1994-03-31,-0.043599


In [45]:
#
pnl_bench=pd.concat([df4,ret_sp],axis="columns", join="outer")
pnl_bench=pnl_bench.rename(columns={0: "MLStrategyPnL"})
pnl_bench[pnl_bench.isnull()] = 0   # nan
pnl_bench = pnl_bench[pnl_bench.MLStrategyPnL != 0]
pnl_bench.tail(10)
pnl_bench=(1 + pnl_bench).cumprod() 
pnl_bench.head(15)
#type(pnl)
#list(pnl.columns) 
#pnl_bench.plot(title='Pnl')
pnl_bench.hvplot.line( title='Pnl', width=1000, height=400)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [46]:
#log(pnl_bench).plot(title='Pnl - logarithmic scale')
log(pnl_bench).hvplot.line( title='Pnl - logarithmic scale', width=1000, height=400)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [47]:
#Calculating Compounded annual growth rate of the ML model compared to SP500 index 
CAGR_ML = ((pnl_bench.iloc[-1,0]/1)**(1/20.666)-1)*100
print("CAGR ML model")
print(CAGR_ML)
CAGR_sp500 = ((pnl_bench.iloc[-1,1]/1)**(1/20.666)-1)*100
print("CAGR SP500 benchmark")
print(CAGR_sp500)


CAGR ML model
12.274006562368612
CAGR SP500 benchmark
6.180761731740314


In [48]:
feat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.060014,0.0,0.081026,0.0,0.152345,0.045494,0.094555,0.067083,0.000000,0.000000,0.013361,0.034252,0.117956,0.061586,0.0,0.078736,0.030982,0.056362,0.106249
1,0.103091,0.0,0.088540,0.0,0.113170,0.054860,0.044524,0.047001,0.038724,0.022665,0.051232,0.031870,0.037150,0.096672,0.0,0.083496,0.049379,0.049811,0.087814
2,0.069413,0.0,0.060691,0.0,0.098732,0.033527,0.054219,0.087037,0.039331,0.032124,0.036502,0.015136,0.054296,0.063858,0.0,0.084524,0.099090,0.085845,0.085676
3,0.067372,0.0,0.077128,0.0,0.099048,0.048631,0.076201,0.066793,0.062655,0.069391,0.013722,0.028970,0.051084,0.104007,0.0,0.059155,0.036565,0.080034,0.059243
4,0.065116,0.0,0.091360,0.0,0.040531,0.070215,0.084785,0.057062,0.031557,0.085737,0.048575,0.038910,0.038248,0.087373,0.0,0.053886,0.083177,0.072648,0.050821


In [49]:
feat=-feat
feature_importance=feat.rank(axis=1)
#feature_importance=feature_importance.max(axis=1).to_frame()

feature_importance.columns=['feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15','feature_16','feature_17','feature_18','feature_19']
feature_importance.head()

feature_importance=feature_importance.mean().rank().to_frame()
feature_importance.columns=['FeatureImportanceRank']
feature_importance=feature_importance.sort_values(by=['FeatureImportanceRank'])
feature_importance.head(20)

,FeatureImportanceRank
feature_14,1.0
feature_19,2.0
feature_13,3.0
feature_18,4.0
feature_11,5.0
feature_3,6.0
feature_7,7.0
feature_5,8.0
feature_12,9.0
feature_17,10.5


In [50]:
print(k)

NameError: name 'k' is not defined

In [ ]:
#rs = pd.np.divide(rank_f, df)
#rs

In [ ]:
print(rank_f.iloc[0].min())

In [ ]:
#rank_f=s_feat10.rank(axis=1)
#rank_f.head()

In [ ]:
rank_f=rank_f.max(axis=1).to_frame()
rank_f.head(2)

In [ ]:
df=(rank_f[[col for col in rank_f.columns for i in range(np.size(s_feat10,1))]])

In [ ]:
print(np(np.repeat(rank_f.columns.values,2)))

In [ ]:
np.size(s_feat10,1)

In [ ]:
rank_f=s_feat10.rank(axis=1)/np.matlib.repmat(s_feat10.max(axis=1), 1, np.size(s_feat10,0))
#/sum(~np.isnan(rank_f))
#rank_f.head()
#print(rank_f.iloc[0].max())
#rank_f.max(axis=1)

In [ ]:
s_feat10.max(axis=1)
df=np.tile(s_feat10.max(axis=1),(1,np.size(s_feat10,0)))
print(df)

In [ ]:
rank_f=s_feat10.rank(axis=0)
df=rank_f.max()
df.head()

In [ ]:
df=s_feat10.max(axis=1)
number_repeats = 50
new_df = df.reindex(np.tile(df.index, number_repeats))
new_df.head()


In [ ]:
np.matlib.repmat(rank_f.max(axis=1), 1, np.size(rank_f,0))

In [ ]:
sum(~np.isnan(rank_f))

In [ ]:
sum(~np.isnan(rank_f.iloc[0]))

In [ ]:
# Reading price data from indices 
temp_csv = Path("Resources/prices.csv")
df_price=pd.read_csv(temp_csv)
df_price.set_index(pd.to_datetime(df_price['Date'], infer_datetime_format=True), inplace=True)
df_price.drop(columns=['Date'], inplace=True)
df_price.head()

In [ ]:
# Reading volume data from indices 
temp_csv = Path("Resources/volume.csv")
df_volume=pd.read_csv(temp_csv)
df_volume.set_index(pd.to_datetime(df_volume['Date'], infer_datetime_format=True), inplace=True)
df_volume.drop(columns=['Date'], inplace=True)
df_volume.head()

In [ ]:
# Reading price data from indices 
temp_csv = Path("Resources/mkt_cap.csv")
df_mktcap=pd.read_csv(temp_csv)
df_mktcap.set_index(pd.to_datetime(df_mktcap['Date'], infer_datetime_format=True), inplace=True)
df_mktcap.drop(columns=['Date'], inplace=True)
df_mktcap.head()

In [ ]:
#Checkig completeness of data so we can set a proper starting point 
msno.matrix(df_mktcap)

In [ ]:
#Checkig completeness of data so we can set a proper starting point 
msno.matrix(df_price)

In [ ]:
df_price_clean=df_price[1306:]
df_price_clean.drop(columns=['XLRE US Equity'], inplace=True)
df_price_clean.dropna(inplace=True)
df_price_clean.head()

In [ ]:
df_mkt_clean=df_mktcap[1306:]
df_mkt_clean.drop(columns=['XLRE US Equity'], inplace=True)
df_mkt_clean.dropna(inplace=True)
df_mkt_clean.head()

In [ ]:
#Calculating sector returns 
ret_sector = df_price_clean.pct_change()
ret_sector = ret_sector.dropna(how='all')
ret_sector.head(3)

In [ ]:
# Reading price data from indices 
temp_csv = Path("Resources/flow.csv")
df_fundflow=pd.read_csv(temp_csv)
df_fundflow.set_index(pd.to_datetime(df_fundflow['Date'], infer_datetime_format=True), inplace=True)
df_fundflow.drop(columns=['Date'], inplace=True)
df_fundflow=df_fundflow.dropna(how='all')
df_fundflow.drop(columns=['XLRE US Equity'], inplace=True)
df_fundflow.head()

In [ ]:
##Building feature set 
window_year=252
window_month=21
window_quarter=63

feat1=df_price_clean/df_price_clean.shift(window_year)-1 #Price momentum 12M 
feat2=df_price_clean.shift(window_month)/df_price_clean.shift(window_year)-1 #Price momentum 12M-1M 
feat3=df_price_clean/df_price_clean.rolling(window_year).max() #Price to 52W High 
feat4=df_price_clean/df_price_clean.rolling(window_year).min() #Price to 52W low 
feat5=df_price_clean/df_price_clean.shift(window_month)-1 #Return 1M
feat6=df_price_clean/df_price_clean.shift(window_quarter)-1 #Return 1Q
feat7=df_price_clean/df_price_clean.shift(window_year*5)-1 #Return 60M
feat8=ret_sector.rolling(window_month).max() #Maximum daily return over 1M
feat9=ret_sector.rolling(window_month).std() #Realized vol 1M
feat10=ret_sector.rolling(window_quarter).std() #Realized vol 3M
feat11=np.log(df_mkt_clean) #Log Market capitalization - already monthly
feat12=df_fundflow.copy() #Fund Flow - already monthly
feat12.tail()

In [ ]:
##Building feature set - monthly features 
feat1.reset_index(inplace=True)
feat1.Date = pd.to_datetime(feat1.Date)
feat1=feat1.resample('m', on='Date').last()
feat1.drop(columns=['Date'], inplace=True)
feat1.dropna(inplace=True)
feat1.head()
#feature
feat2.reset_index(inplace=True)
feat2.Date = pd.to_datetime(feat2.Date)
feat2=feat2.resample('m', on='Date').last()
feat2.drop(columns=['Date'], inplace=True)
feat2.dropna(inplace=True)
feat2.head()
#feature
feat3.reset_index(inplace=True)
feat3.Date = pd.to_datetime(feat3.Date)
feat3=feat3.resample('m', on='Date').last()
feat3.drop(columns=['Date'], inplace=True)
feat3.dropna(inplace=True)
feat3.head()
#feature
feat4.reset_index(inplace=True)
feat4.Date = pd.to_datetime(feat4.Date)
feat4=feat4.resample('m', on='Date').last()
feat4.drop(columns=['Date'], inplace=True)
feat4.dropna(inplace=True)
feat4.head()
#feature
feat5.reset_index(inplace=True)
feat5.Date = pd.to_datetime(feat5.Date)
feat5=feat5.resample('m', on='Date').last()
feat5.drop(columns=['Date'], inplace=True)
feat5.dropna(inplace=True)
feat5.head()
#feature
feat6.reset_index(inplace=True)
feat6.Date = pd.to_datetime(feat6.Date)
feat6=feat6.resample('m', on='Date').last()
feat6.drop(columns=['Date'], inplace=True)
feat6.dropna(inplace=True)
feat6.head()
#feature
feat7.reset_index(inplace=True)
feat7.Date = pd.to_datetime(feat7.Date)
feat7=feat7.resample('m', on='Date').last()
feat7.drop(columns=['Date'], inplace=True)
feat7.dropna(inplace=True)
feat7.head()
#feature
feat8.reset_index(inplace=True)
feat8.Date = pd.to_datetime(feat8.Date)
feat8=feat8.resample('m', on='Date').last()
feat8.drop(columns=['Date'], inplace=True)
feat8.dropna(inplace=True)
feat8.head()
#feature
feat9.reset_index(inplace=True)
feat9.Date = pd.to_datetime(feat9.Date)
feat9=feat9.resample('m', on='Date').last()
feat9.drop(columns=['Date'], inplace=True)
feat9.dropna(inplace=True)
feat9.head()
#feature
feat10.reset_index(inplace=True)
feat10.Date = pd.to_datetime(feat10.Date)
feat10=feat10.resample('m', on='Date').last()
feat10.drop(columns=['Date'], inplace=True)
feat10.dropna(inplace=True)
feat10.head()



In [ ]:
# monthly features here 123
#s_feature
s_feat1.reset_index(inplace=True)
s_feat1.Date = pd.to_datetime(s_feat1.Date)
s_feat1=s_feat1.resample('m', on='Date').last()
s_feat1.drop(columns=['Date'], inplace=True)
s_feat1.dropna(how='all',inplace=True)
s_feat1.head()
#s_feature
s_feat2.reset_index(inplace=True)
s_feat2.Date = pd.to_datetime(s_feat2.Date)
s_feat2=s_feat2.resample('m', on='Date').last()
s_feat2.drop(columns=['Date'], inplace=True)
s_feat2.dropna(how='all',inplace=True)
s_feat2.head()
#s_feature
s_feat3.reset_index(inplace=True)
s_feat3.Date = pd.to_datetime(s_feat3.Date)
s_feat3=s_feat3.resample('m', on='Date').last()
s_feat3.drop(columns=['Date'], inplace=True)
s_feat3.dropna(how='all',inplace=True)
s_feat3.head()
#s_feature
s_feat4.reset_index(inplace=True)
s_feat4.Date = pd.to_datetime(s_feat4.Date)
s_feat4=s_feat4.resample('m', on='Date').last()
s_feat4.drop(columns=['Date'], inplace=True)
s_feat4.dropna(how='all',inplace=True)
s_feat4.head()
#s_feature
s_feat5.reset_index(inplace=True)
s_feat5.Date = pd.to_datetime(s_feat5.Date)
s_feat5=s_feat5.resample('m', on='Date').last()
s_feat5.drop(columns=['Date'], inplace=True)
s_feat5.dropna(how='all',inplace=True)
s_feat5.head()
#s_feature
s_feat6.reset_index(inplace=True)
s_feat6.Date = pd.to_datetime(s_feat6.Date)
s_feat6=s_feat6.resample('m', on='Date').last()
s_feat6.drop(columns=['Date'], inplace=True)
s_feat6.dropna(how='all',inplace=True)
##Building s_feature set - monthly s_features 
s_feat1.reset_index(inplace=True)
s_feat1.Date = pd.to_datetime(s_feat1.Date)
s_feat1=s_feat1.resample('m', on='Date').last()
s_feat1.drop(columns=['Date'], inplace=True)
s_feat1.dropna(how='all',inplace=True)
s_feat6.head()
#s_feature
s_feat7.reset_index(inplace=True)
s_feat7.Date = pd.to_datetime(s_feat7.Date)
s_feat7=s_feat7.resample('m', on='Date').last()
s_feat7.drop(columns=['Date'], inplace=True)
s_feat7.dropna(how='all',inplace=True)
s_feat7.head()
#s_feature
s_feat8.reset_index(inplace=True)
s_feat8.Date = pd.to_datetime(s_feat8.Date)
s_feat8=s_feat8.resample('m', on='Date').last()
s_feat8.drop(columns=['Date'], inplace=True)
s_feat8.dropna(how='all',inplace=True)
s_feat8.head()
#s_feature
s_feat9.reset_index(inplace=True)
s_feat9.Date = pd.to_datetime(s_feat9.Date)
s_feat9=s_feat9.resample('m', on='Date').last()
s_feat9.drop(columns=['Date'], inplace=True)
s_feat9.dropna(how='all',inplace=True)
s_feat9.head()
#s_feature
s_feat10.reset_index(inplace=True)
s_feat10.Date = pd.to_datetime(s_feat10.Date)
s_feat10=s_feat10.resample('m', on='Date').last()
s_feat10.drop(columns=['Date'], inplace=True)
s_feat10.dropna(how='all',inplace=True)
s_feat10.head()